In [34]:
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import numpy as np

# Load sentences from the text file
def load_sentences_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    sentences = [line.strip().split(': ')[1] for line in lines]
    return sentences

# Calculate BLEU score
def calculate_bleu_scores(file_sentences, df_sentences):
    # references = [[ref.lower().split()] for ref in df_sentences]
    # hypotheses = [[sent.lower().split()] for sent in file_sentences]
    # bleu_score = sentence_bleu(references, hypotheses)

    bleu_scores = []
    for ref, sent in tqdm(zip(df_sentences, file_sentences)):
        try:
            bleu_scores.append(sentence_bleu(ref.lower().split(), sent, smoothing_function=SmoothingFunction().method4))
        except:
            print("problem")

    bleu_score = np.array(bleu_scores).mean()
    return bleu_score

In [35]:
 # Load DataFrame
# Assuming df_sentences is your DataFrame column containing the sentences
test_data = pd.read_csv("./data/test_replics.csv")
def remove_teacher(text):
    return text[10:]

test_data = test_data["text"].apply(remove_teacher)

# Path to your text file
sentences_file = 'predicted_answers_without_label.txt'  # Change 'your_sentences.txt' to your file path

In [36]:
test_data

0       Okay -  you've overcomplicated this. Let's sta...
1       No, you need to add as she still has the spoon...
2                                        Yes, that's it\n
3       Okay, let's start again. If each bid adds $50 ...
4       Yes $150. Carmen adds a $50 bid after each of ...
                              ...                        
3095                 3*60+40+70 = 180+40+70=290 minutes\n
3096    That doesn't seem to be correct. Can you find ...
3097    Great. The question now says that 1/5th of tha...
3098    That's wonderful. So now what do you think how...
3099            There's your answer. I hope you get it.\n
Name: text, Length: 3100, dtype: object

In [37]:
import pandas as pd
import re

def read_file_to_dataframe(file_path):
    data = []
    current_index = None
    current_utterance = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Check if the line starts with an index pattern (number followed by a colon)
            if re.match(r'^\d+:', line):
                # If there's an existing utterance, save it before starting a new one
                if current_index is not None:
                    data.append([current_index, ''.join(current_utterance).strip()])
                # Start a new utterance
                current_index, rest_of_line = line.split(':', 1)
                current_utterance = [rest_of_line]
            else:
                # Continue accumulating lines for the current utterance
                current_utterance.append(line)
    
    # Add the last utterance to the data if it exists
    if current_index is not None:
        data.append([current_index, ''.join(current_utterance).strip()])

    # Create DataFrame
    df = pd.DataFrame(data, columns=['Index', 'Utterance'])
    return df

# Specify the path to your file
file_path = 'predicted_answers_focus_label.txt'
df = read_file_to_dataframe(file_path)
print(df["Utterance"])

file_sentences = df["Utterance"]


0       Great job, Mariana! Your solution is clear and...
1       Teacher: Great, so Julia had 9 spoons left whe...
2       Great job! You got the correct answer in the e...
3       Great job, Ayisha! You correctly identified th...
4       Student: So, if the initial bid was $200, and ...
                              ...                        
3095    Teacher: Great job! You corrected the mistake ...
3096    Your solution is correct and well thought out....
3097            Student: So, 225 cars have valid tickets.
3098    Teacher: Good job! Now, can you determine how ...
3099    Great job! You have successfully calculated th...
Name: Utterance, Length: 3100, dtype: object


In [38]:
# Calculate BLEU score
avg_bleu_score = calculate_bleu_scores(list(file_sentences), list(test_data))

# Print average BLEU score
print(f"Average BLEU score: {avg_bleu_score}")


1892it [00:05, 357.74it/s]

problem
problem


2599it [00:07, 325.88it/s]

problem
problem
problem
problem
problem
problem
problem
problem


3100it [00:08, 353.80it/s]

problem
problem
problem
Average BLEU score: 0.051594120446071395


In [39]:

from bert_score import score

# Calculate BERTScore
def calculate_bert_scores(file_sentences, df_sentences):
    _, _, bert_scores = score(file_sentences, df_sentences, lang="en", verbose=True)
    print(bert_scores.shape)
    avg_bert_score = bert_scores.mean().item()
    return avg_bert_score

    # Calculate BERTScore

avg_bert_score = calculate_bert_scores(list(file_sentences), list(test_data))

print(avg_bert_score)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/95 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/49 [00:00<?, ?it/s]

done in 7.82 seconds, 396.46 sentences/sec
torch.Size([3100])
0.8458389043807983
